<h1>Final Capstone Project: Hockey Clubs in Belgium. Where do they thrive best? </h1>

<p>Field Hockey is a booming sport in Belgium, with our national hockey team being nr 1 in the world.  New clubs are being opened.  

<br>How is the distribution of hockey clubs across Flanders?
<br>By plotting the Flanders communes and the hockeyclubs in their proximity, we can visualize where there is potential for starting a new club.
<br>Using foursquare data we can cluster the existing hockey clubs, and see which cluster characteristics would suit the new potential locations.

<br>The following data will be used
<br> Excel file downloaded from www including all hockey clubs in Flanders/Belgium with their address and number of members
<br> Wikipedia list of Flanders communes.   Using screenscraping to use in dataframe
<br> Foursquare API to list all venues in neighbourhood of these hockey clubs

By analysing this data we can determine how the environment where a club is built can contribute to success (number of members,  competitive quality)
 </p>

<h1>Notebook preparation</h1>

In [4]:
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes
!conda install lxml
!conda install requests

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [9]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library


In [10]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [11]:
from bs4 import BeautifulSoup
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

<h1>Flanders communes</h1>

<h2>Display Flanders communes on map</h2>

In [12]:
#Extract communes from wikipedia,  use Beautifulsoup, and read to dataframe
website_URL=requests.get('https://nl.wikipedia.org/wiki/Lijst_van_gemeenten_in_het_Vlaams_Gewest').text
soup = BeautifulSoup(website_URL,'lxml')
#print(soup.prettify())
My_table = soup.find('table',{'class':'wikitable sortable'}).prettify()
dfs = pd.read_html(My_table)
df=dfs[0]
df.head()

,Gemeente,Stad/ Gem.,Provincie,Opper- vlakte (km²),Aantal inwoners (01/01/2016),Deelgemeenten
0,Aalst,stad,Oost-Vlaanderen,7812,84.329,"Aalst, Gijzegem , Hofstade , Baardegem ,..."
1,Aalter [1],gem.,Oost-Vlaanderen,11919,28.435,"Aalter, Bellem , Knesselare , Lotenhulle ..."
2,Aarschot,stad,Vlaams-Brabant,6252,29.529,"Aarschot, Gelrode , Langdorp , Rillaar"
3,Aartselaar,gem.,Antwerpen,1093,14.262,Aartselaar
4,Affligem,gem.,Vlaams-Brabant,1770,13.079,"Essene , Hekelgem , Teralfene"


In [14]:
#Using geopy to add longitute and lattitude data to plot on map 
#Add coordinates and clean up
geolocator = Nominatim(user_agent="foursquare_agent")

from geopy.extra.rate_limiter import RateLimiter
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['Gemeente'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

df_clean = df.drop(["point", "altitude", "location"], axis=1)
#Drop NA records and sort
#df_clean = df_clean.dropna()
df_clean = df_clean.sort_values(by=df_clean.columns[4], ascending=False)
df_clean.shape

,Gemeente,Stad/ Gem.,Provincie,Opper- vlakte (km²),Aantal inwoners (01/01/2016),Deelgemeenten,latitude,longitude
7,Antwerpen,stad,Antwerpen,20451,517.042,"Antwerpen , Berendrecht-Zandvliet-Lillo , ...",51.221110,4.399708
75,Gent,stad,Oost-Vlaanderen,15618,257.029,"Gent, Mariakerke , Drongen , Wondelgem ,...",51.053829,3.725012
45,Brugge,stad,West-Vlaanderen,13840,118.053,"Brugge, Koolkerke , Sint-Andries , Sint-M...",51.208553,3.226772
156,Leuven,stad,Vlaams-Brabant,5663,99.288,"Leuven, Heverlee , Kessel-Lo , Wilsele ,...",50.879202,4.701168
102,Herstappe,gem. [2],Limburg,135,89.000,Herstappe,50.727383,5.426092


In [57]:
df_clean.head(200)

,Gemeente,Stad/ Gem.,Provincie,Opper- vlakte (km²),Aantal inwoners (01/01/2016),Deelgemeenten,latitude,longitude
indcol,,,,,,,,
7,Antwerpen,stad,Antwerpen,20451,517.042,"Antwerpen , Berendrecht-Zandvliet-Lillo , ...",51.221110,4.399708
75,Gent,stad,Oost-Vlaanderen,15618,257.029,"Gent, Mariakerke , Drongen , Wondelgem ,...",51.053829,3.725012
45,Brugge,stad,West-Vlaanderen,13840,118.053,"Brugge, Koolkerke , Sint-Andries , Sint-M...",51.208553,3.226772
156,Leuven,stad,Vlaams-Brabant,5663,99.288,"Leuven, Heverlee , Kessel-Lo , Wilsele ,...",50.879202,4.701168
102,Herstappe,gem. [2],Limburg,135,89.000,Herstappe,50.727383,5.426092
179,Mechelen,stad,Antwerpen,6519,84.523,"Mechelen, Walem , Heffen , Hombeek , Le...",51.028138,4.480345
0,Aalst,stad,Oost-Vlaanderen,7812,84.329,"Aalst, Gijzegem , Hofstade , Baardegem ,...",50.938323,4.039215
91,Hasselt,stad,Limburg,10224,76.685,"Hasselt, Sint-Lambrechts-Herk , Wimmertinge...",50.930374,5.337804
140,Kortrijk,stad,West-Vlaanderen,8002,75.506,"Kortrijk, Bissegem , Heule , Bellegem , ...",50.827643,3.265988


In [15]:
#map of Flanders with Brussels as center
address = 'Brussels, Belgium'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Flanders with Brussels in center, using latitude and longitude values
map_flanders = folium.Map(location=[latitude, longitude], zoom_start=100)

# add markers to map
for lat, lng, gemeente in zip(df_clean['latitude'], df_clean['longitude'], df_clean['Gemeente']):
    label = '{}'.format(gemeente)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_flanders)  
    
map_flanders

<h1>Hockey clubs in Flanders</h1>

In [16]:
# The code was removed by Watson Studio for sharing.

In [17]:
#geopy for geo data for mapping
from geopy.extra.rate_limiter import RateLimiter
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# 2- - create location column
df_data_1['location'] = df_data_1['Adres'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df_data_1['point'] = df_data_1['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df_data_1[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_data_1['point'].tolist(), index=df_data_1.index)
#df_data_1.head()


In [18]:
#more clean up
# Delete multiple columns from the dataframe
df_data_clean = df_data_1.drop(["point", "altitude", "location"], axis=1)
#Sort on number of members "Leden"
df_data_clean = df_data_clean.sort_values(by="Leden", ascending=False)
#Drop NA records
df_data_clean = df_data_clean.dropna()
#df_data_clean.head(50)

In [19]:
#select top X biggest clubs
df_data_clean_top5=df_data_clean.head(5)
df_data_clean_top5.head(5)

,Club,Website,Adres,E-mail,Hockey,Leden,latitude,longitude
46,R Victory HC,www.victory.be,"Kattenbroek 5, 2650 Edegem, België",info@victory.be,Recr. Comp.,1589.0,51.149110,4.430133
30,KHC Dragons,www.dragons.be,"Gemeentepark 12, 2930 Brasschaat, België",secretaris@dragons.be,Recr. Comp.,1426.0,51.284169,4.488786
0,ARA Gantoise HC,www.gantoise.be,"Tennisstraat 13, 9050 Gent, België",hockey@gantoise.be,Recr. Comp.,1349.0,51.030502,3.766894
3,Braxgata HC,www.braxgata.be,"Schommelei 1-4, 2850 Boom, België",secretaris@braxgata.be,Recr. Comp.,1290.0,51.089335,4.375491
26,Indiana THC,www.thc-indiana.be,"Hondelestraat 13,9840 De Pinte, België",hockey@thc-indiana.be,Recr. Comp.,1103.0,50.993195,3.693569


<h2>Map hockey clubs on the Flanders commune map</h2>

In [ ]:
#Plot biggest club as center of the map

address = 'Kattenbroek 5, 2650 Edegem, België'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of HC Victory are {}, {}.'.format(latitude, longitude))

In [20]:
# create map of Flanders with HC Victory in center, using latitude and longitude values
map_victory = folium.Map(location=[latitude, longitude], zoom_start=100)

# add markers to map
for lat, lng, club in zip(df_data_clean['latitude'], df_data_clean['longitude'], df_data_clean['Club']):
    label = '{}'.format(club)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#FFFF00',
        fill_opacity=0.7,
        parse_html=False).add_to(map_flanders)  
    
map_flanders

<h1>Intermediate conclusion</h1>

Hockey clubs are spread over Flanders, but there are a few areas where no hockey club is established, and where we would expect potential for starting one.

Building a new hockey club is expensive.  A hockey field costs over 400.000 euro.  Sponsorship of the local commune/city is inevitable.
As the local authorities are inclined to invest only where there is an existing sports infrastructure, let's use foursquare to list the "sport facilities" in the communes where there are no nearby hockey clubs.


<h2>Use Foursquare data to find sports facilities in communes where no hockey clubs are present</h2>

In [21]:
#Let's look at the first possible location.
df_clean.loc[7, 'Gemeente']

'Antwerpen'

In [59]:
#List cities in areas with no hockey presence
#visually identified Brugge, Dendermonde as 2 relatively large cities with no hockey-club nearby.
#Let's create a dataframe with these cities and then plot sportsvenues
df_nohockey = df_clean.loc[ [45 , 54] , : ]
df_nohockey.head()

,Gemeente,Stad/ Gem.,Provincie,Opper- vlakte (km²),Aantal inwoners (01/01/2016),Deelgemeenten,latitude,longitude
indcol,,,,,,,,
45,Brugge,stad,West-Vlaanderen,13840,118.053,"Brugge, Koolkerke , Sint-Andries , Sint-M...",51.208553,3.226772
54,Dendermonde,stad,Oost-Vlaanderen,5567,45.367,"Dendermonde, Appels , Baasrode , Gremberg...",51.031229,4.098112


In [60]:
#Starting with Bruge
neighborhood_latitude = df_clean.loc[45, 'latitude'] # neighborhood latitude value
neighborhood_longitude = df_clean.loc[45, 'longitude'] # neighborhood longitude value

neighborhood_name = df_clean.loc[45, 'Gemeente'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Brugge are 51.2085526, 3.226772.


In [61]:
CLIENT_ID = 'ZLSHLAT3WKZAQPYSDTLJLIV12HXGEOPK4LK20HYAYQHBMW23' # your Foursquare ID
CLIENT_SECRET = 'ZAGZ0YF14LUYY3LABPRRM4ID0RVZN4HCD2OE1KWSHMXES0OY' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZLSHLAT3WKZAQPYSDTLJLIV12HXGEOPK4LK20HYAYQHBMW23
CLIENT_SECRET:ZAGZ0YF14LUYY3LABPRRM4ID0RVZN4HCD2OE1KWSHMXES0OY


In [93]:
# let's get the sports facilities within  10 kilometer of Bruge

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
search_query='Athletics & Sports'
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    search_query,
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZLSHLAT3WKZAQPYSDTLJLIV12HXGEOPK4LK20HYAYQHBMW23&client_secret=ZAGZ0YF14LUYY3LABPRRM4ID0RVZN4HCD2OE1KWSHMXES0OY&v=20180604&ll=51.2085526,3.226772&query=Athletics & Sports&radius=10000&limit=100'

In [94]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5da9b85df9dbde0038833245'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bruges',
  'headerFullLocation': 'Bruges',
  'headerLocationGranularity': 'city',
  'query': 'athletics',
  'totalResults': 195,
  'suggestedBounds': {'ne': {'lat': 51.298552690000086,
    'lng': 3.370162017481249},
   'sw': {'lat': 51.11855250999991, 'lng': 3.083381982518751}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50598b18e4b0e107f9c60548',
       'name': 'Basic-Fit',
       'location': {'address': 'Maalse Steenweg 50',
        'lat': 51.21094859876742,
        'lng': 3.2432413013385673,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.2109

<h2>Write function to get sports facilities for all communes with no hockey presence</h2>

In [95]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    search_query='sports'
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            search_query,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Club', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [96]:
hockey_venues = getNearbyVenues(names=df_nohockey['Gemeente'],
                                   latitudes=df_clean['latitude'],
                                   longitudes=df_clean['longitude']
                                  )

Brugge
Dendermonde


In [97]:
print(hockey_venues.shape)
hockey_venues.head(200)

(42, 7)


,Club,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Brugge,51.221110,4.399708,Decathlon,51.246997,4.417184,Sporting Goods Shop
1,Brugge,51.221110,4.399708,Addict Store,51.215809,4.402172,Sporting Goods Shop
2,Brugge,51.221110,4.399708,Lotto Arena,51.230260,4.441824,Music Venue
3,Brugge,51.221110,4.399708,Addict Store,51.200197,4.442083,Sporting Goods Shop
4,Brugge,51.221110,4.399708,Provinciaal Groendomein Rivierenhof,51.217211,4.468476,State / Provincial Park
5,Brugge,51.221110,4.399708,Sporthal Parkloods,51.230328,4.426701,Athletics & Sports
6,Brugge,51.221110,4.399708,M4teria,51.205566,4.429027,Gym
7,Brugge,51.221110,4.399708,Stadspark,51.211920,4.414652,Park
8,Brugge,51.221110,4.399708,David Lloyd Antwerpen,51.149955,4.431470,Gym / Fitness Center
9,Brugge,51.221110,4.399708,Radisson Blu Astrid Hotel,51.219006,4.421202,Hotel


In [98]:
#plot sportsfacilities on the Flanders map
# create map of Flanders with Brussels in center, using latitude and longitude values
map_sportsfac = folium.Map(location=[latitude, longitude], zoom_start=100)

# add markers to map
for lat, lng, club in zip(hockey_venues['Venue Latitude'], hockey_venues['Venue Longitude'], hockey_venues['Club']):
    label = '{}'.format(club)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='white',
        fill=True,
        fill_color='#FFF000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_flanders)  
    
map_flanders

<h1>Conclusion</h1>

Considering the 2 locations we analyzed, it appears Bruge has the most Athletics & Sports venues so is more elegible to receive support from the local city to get sponsoring for a new hockey field.
This analysis is based on incomplete data for the purpose of this project, but should not be used for any decision making.
